In [9]:


# ________________Manipulation_____________
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from itertools import product
import matplotlib.pyplot as plt

# Importing Data

In [2]:
import os

# Local version of the same logic
DATASET_PATH = "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw"

print("Listing files in:", DATASET_PATH)
for dirname, _, filenames in os.walk(DATASET_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Listing files in: C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\items.csv
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\item_categories.csv
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\sales_train.csv
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\sample_submission.csv
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\shops.csv
C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw\test.csv


In [6]:
# Importing  the different dataframes
df_sales_train = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/sales_train.csv",
    parse_dates=["date"],
)
df_sales_test = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/test.csv"
)
df_sample_submission = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/sample_submission.csv"
)
df_item_info = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/items.csv"
)
df_item_cat = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/item_categories.csv"
)
df_shop_info = pd.read_csv(
    "C:/Users/LENOVO/Desktop/Sales Forecasing/data/raw/shops.csv"
)

In [10]:
df_sales_test.head(3)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233


In [11]:
print(df_sales_train.shape)
df_sales_train.head(3)

(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0


In [12]:
print(df_item_info.shape)
df_item_info.head(3)

(22170, 3)


,item_name,item_id,category_id
0,!! IN THE POWER OF HAPPINESS (PLAST) D,0,40
1,! ABBYY FineReader 12 Professional Edition Ful...,1,76
2,*** IN THE GLORY OF THE GLORY (UNV) D,2,40


In [13]:
print(df_item_cat.shape)
df_item_cat.head(3)

(84, 2)


,item_category_name,item_category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2


In [14]:
print(df_shop_info.shape)
df_shop_info.head(3)

(60, 2)


,shop_name,shop_id
0,"! Yakutsk Ordzhonikidze, 56 francs",0
1,"! Yakutsk TC ""Central"" fran",1
2,"Adygea TC ""Mega""",2


# Join all Dataframes

In [15]:
# Creating the Full Dataset
print("The Full Dataframe is the following")
df_sales_train1 = df_sales_train.join(
    other=df_shop_info, on="shop_id", how="inner", rsuffix="_"
).drop(
    columns="shop_id_"
)  # we add rsuffix, because when it's not like merge, here it adds the shop_id twice,and if we don't add the suffix it we will have 2 columns wit the same
df_sales_train1 = df_sales_train1.join(
    other=df_item_info, on="item_id", how="inner", rsuffix="_"
).drop(columns="item_id_")

df_sales_train1 = df_sales_train1.rename(
    columns={"category_id": "item_category_id"}
)
df_sales_train1 = df_sales_train1.join(
    other=df_item_cat, on="item_category_id", how="inner", rsuffix="_"
).drop(columns="item_category_id_")
df_sales_train1

The Full Dataframe is the following


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
0,02.01.2013,0,59,22154,999.00,1.0,"Yaroslavl TC"" Altair """,SCENE 2012 (BD),37,Cinema - Blu-Ray
1,03.01.2013,0,25,2552,899.00,1.0,"Moscow TEC"" Atrium """,DEEP PURPLE The House Of Blue Light LP,58,Music - Vinyl
2,05.01.2013,0,25,2552,899.00,-1.0,"Moscow TEC"" Atrium """,DEEP PURPLE The House Of Blue Light LP,58,Music - Vinyl
3,06.01.2013,0,25,2554,1709.05,1.0,"Moscow TEC"" Atrium """,DEEP PURPLE Who Do You Think We Are LP,58,Music - Vinyl
4,15.01.2013,0,25,2555,1099.00,1.0,"Moscow TEC"" Atrium """,DEEP PURPLE 30 Very Best Of 2CD (Businesses).,56,Music - CD of corporate production
...,...,...,...,...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0,"Moscow TEC"" Atrium """,V/A Nu Jazz Selection (digipack),55,Music - CD of local production
2935845,09.10.2015,33,25,7460,299.00,1.0,"Moscow TEC"" Atrium """,V/A The Golden Jazz Collection 1 2CD,55,Music - CD of local production
2935846,14.10.2015,33,25,7459,349.00,1.0,"Moscow TEC"" Atrium """,V/A The Best Of The 3 Tenors,55,Music - CD of local production
2935847,22.10.2015,33,25,7440,299.00,1.0,"Moscow TEC"" Atrium """,V/A Relax Collection Planet MP3 (mp3-CD) (jewel),57,Music - MP3


In [16]:
df_sales_train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   date_block_num      int64  
 2   shop_id             int64  
 3   item_id             int64  
 4   item_price          float64
 5   item_cnt_day        float64
 6   shop_name           object 
 7   item_name           object 
 8   item_category_id    int64  
 9   item_category_name  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 224.0+ MB


In [17]:
df_sales_train1.describe().T

,count,mean,std,min,25%,50%,75%,max
date_block_num,2935849.0,14.569911,9.422988,0.0,7.0,14.0,23.0,33.0
shop_id,2935849.0,33.001728,16.226973,0.0,22.0,31.0,47.0,59.0
item_id,2935849.0,10197.227057,6324.297354,0.0,4476.0,9343.0,15684.0,22169.0
item_price,2935849.0,890.853233,1729.799631,-1.0,249.0,399.0,999.0,307980.0
item_cnt_day,2935849.0,1.242641,2.618834,-22.0,1.0,1.0,1.0,2169.0
item_category_id,2935849.0,40.001383,17.100759,0.0,28.0,40.0,55.0,83.0


# Preprocessing 

In [18]:
print(len(df_sales_train1[df_sales_train1['item_cnt_day']<0]))
df_sales_train1[df_sales_train1['item_cnt_day']<0]

7356


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
2,05.01.2013,0,25,2552,899.0,-1.0,"Moscow TEC"" Atrium """,DEEP PURPLE The House Of Blue Light LP,58,Music - Vinyl
148,23.01.2013,0,25,2321,999.0,-1.0,"Moscow TEC"" Atrium """,Call of Duty: Black Ops II. Extended edition [...,28,PC Games - Additional Editions
175,07.01.2013,0,25,2199,1449.0,-1.0,"Moscow TEC"" Atrium """,COLDPLAY Mylo Xyloto LP,58,Music - Vinyl
807,02.01.2013,0,25,2330,599.0,-1.0,"Moscow TEC"" Atrium ""","Call of Duty: Black Ops [PC, Jewel, Russian ve...",30,PC Games - Standard Editions
1041,13.01.2013,0,25,5034,1989.0,-1.0,"Moscow TEC"" Atrium ""","NHL 13 [Xbox 360, Russian documentation]",23,Games - XBOX 360
...,...,...,...,...,...,...,...,...,...,...
2934243,26.10.2015,33,25,3917,449.0,-1.0,"Moscow TEC"" Atrium """,IRON MAIDEN The Book Of Souls 2CD,55,Music - CD of local production
2934462,18.10.2015,33,25,4896,6398.0,-1.0,"Moscow TEC"" Atrium ""","Mortal Kombat X. Kollector's Edition [PS4, Rus...",20,Games - PS4
2935263,05.10.2015,33,25,10039,249.0,-1.0,"Moscow TEC"" Atrium """,REBELLION OF THE APE PLANET (BD),37,Cinema - Blu-Ray
2935643,16.10.2015,33,25,7893,2990.0,-1.0,"Moscow TEC"" Atrium """,X360: Gamepad Wireless Black - Wireless Contro...,6,Accessories - XBOX 360


There are arround 7000 rows where the item quantity is negative, we will assume these are returns and drop them from the df

In [19]:
df_sales_train1[df_sales_train1['item_price']<0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
484683,15.05.2013,4,32,2973,-1.0,1.0,"Moscow TC"" Serebryany House ""","DmC Devil May Cry [PS3, Russian subtitles]",19,Games - PS3


In [20]:
# Deleting the Returns and the negative amounts 
df=df_sales_train1.copy()
df_sales_train1=df_sales_train1[(df_sales_train1['item_price']>0) & (df_sales_train1['item_cnt_day']>0)]
df_sales_train1

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
0,02.01.2013,0,59,22154,999.00,1.0,"Yaroslavl TC"" Altair """,SCENE 2012 (BD),37,Cinema - Blu-Ray
1,03.01.2013,0,25,2552,899.00,1.0,"Moscow TEC"" Atrium """,DEEP PURPLE The House Of Blue Light LP,58,Music - Vinyl
3,06.01.2013,0,25,2554,1709.05,1.0,"Moscow TEC"" Atrium """,DEEP PURPLE Who Do You Think We Are LP,58,Music - Vinyl
4,15.01.2013,0,25,2555,1099.00,1.0,"Moscow TEC"" Atrium """,DEEP PURPLE 30 Very Best Of 2CD (Businesses).,56,Music - CD of corporate production
5,10.01.2013,0,25,2564,349.00,1.0,"Moscow TEC"" Atrium """,DEEP PURPLE Perihelion: Live In Concert DVD (C...,59,Music - Music video
...,...,...,...,...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0,"Moscow TEC"" Atrium """,V/A Nu Jazz Selection (digipack),55,Music - CD of local production
2935845,09.10.2015,33,25,7460,299.00,1.0,"Moscow TEC"" Atrium """,V/A The Golden Jazz Collection 1 2CD,55,Music - CD of local production
2935846,14.10.2015,33,25,7459,349.00,1.0,"Moscow TEC"" Atrium """,V/A The Best Of The 3 Tenors,55,Music - CD of local production
2935847,22.10.2015,33,25,7440,299.00,1.0,"Moscow TEC"" Atrium """,V/A Relax Collection Planet MP3 (mp3-CD) (jewel),57,Music - MP3


In [21]:
df_sales_train1.isnull().sum()

date                  0
date_block_num        0
shop_id               0
item_id               0
item_price            0
item_cnt_day          0
shop_name             0
item_name             0
item_category_id      0
item_category_name    0
dtype: int64

In [22]:
df_sales_train1["date"] = pd.to_datetime(
    df_sales_train1["date"], format="%d.%m.%Y", dayfirst=True
)
df_sales_train1["week_id"] = df_sales_train1["date"].dt.strftime("%Y-W%U")
df_sales_train1["Revenue"] = (
    df_sales_train1["item_price"] * df_sales_train1["item_cnt_day"]
)
df_sales_train1.head(3)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12720\4216222606.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales_train1["date"] = pd.to_datetime(
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12720\4216222606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales_train1["week_id"] = df_sales_train1["date"].dt.strftime("%Y-W%U")
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12720\4216222606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name,week_id,Revenue
0,2013-01-02,0,59,22154,999.00,1.0,"Yaroslavl TC"" Altair """,SCENE 2012 (BD),37,Cinema - Blu-Ray,2013-W00,999.00
1,2013-01-03,0,25,2552,899.00,1.0,"Moscow TEC"" Atrium """,DEEP PURPLE The House Of Blue Light LP,58,Music - Vinyl,2013-W00,899.00
3,2013-01-06,0,25,2554,1709.05,1.0,"Moscow TEC"" Atrium """,DEEP PURPLE Who Do You Think We Are LP,58,Music - Vinyl,2013-W01,1709.05


# EDA 

## Product based Analysis 

In [23]:
df_prod = df_sales_train1.groupby(
    ["week_id", "item_category_id"], as_index=False
).agg(
    {
        "date": "first",
        "item_cnt_day": "sum",
    }
)

categories = df_prod["item_category_id"].unique()
for cat in categories:
    df1 = df_prod[df_prod["item_category_id"] == cat]
    df1_grouped = (
        df1.groupby("week_id")
        .agg(
            {
                "date": "first",
                "item_cnt_day": "sum",
                "item_category_id": "first",
            }
        )
        .reset_index()
    )

    df1_grouped = df1_grouped.sort_values(by="date", ascending=True)
    fig = px.line(
        df1_grouped,
        x="week_id",
        y="item_cnt_day",
        labels={
            "x": "week_id",
            "y": "item_cnt_day",
        },
        title=f"Qty sold (All shops) of product category {cat}",
        hover_data={"item_category_id": True},
    )
    fig.update_layout(
        xaxis_tickangle=90,
    )
    fig.show()